In [ ]:
!pip install simupy

from google.colab import drive
drive.mount('/content/drive')

import pickle
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Define the function to load variables
def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

# Function to move 'label' column to the first position
def move_label_to_first_column(df):
    label_column = df.pop('label')
    df.insert(0, 'label', label_column)
    return df

# Function to prepare data for a given year
def prepare_data(year):
    loaded_variables = load_variables(year)

    Eora_pp_FD = loaded_variables['Eora_pp_FD']
    labels_FD = loaded_variables['labels_FD']
    Eora_pp_Q = loaded_variables['Eora_pp_Q']
    labels_Q = loaded_variables['labels_Q']
    Eora_pp_QY = loaded_variables['Eora_pp_QY']
    Eora_pp_T = loaded_variables['Eora_pp_T']
    labels_T = loaded_variables['labels_T']
    Eora_pp_VA = loaded_variables['Eora_pp_VA']
    labels_VA = loaded_variables['labels_VA']
    Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
    Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
    xout = loaded_variables['xout']
    A = loaded_variables['A']
    f = loaded_variables['f']
    I = loaded_variables['I']
    L = loaded_variables['L']
    sector_labels = loaded_variables['sector_labels']

    # Add labels to dataframe and move 'label' column to the first position
    for df, col_index, eora in zip([labels_FD, labels_Q, labels_T, labels_VA], [3, 1, 3, 1], [Eora_pp_FD, Eora_pp_Q, Eora_pp_T, Eora_pp_VA]):
        eora['label'] = df.iloc[:, col_index]
        eora = move_label_to_first_column(eora)

    # Extract energy-related labels from T
    energy_condition = Eora_pp_T['label'].str.contains('Electricity, Gas and Water', case=False, na=False)
    energy_Eora_pp_T = Eora_pp_T.loc[energy_condition]
    solar_panels_production = energy_Eora_pp_T # If we want to use it as a proxy for the first tries

    # Extract solar-related emissions from Q
    solar_panels_condition = Eora_pp_Q['label'].str.contains('Solar, Tide and Wave Electricity', case=False, na=False)
    solar_panels_emissions = Eora_pp_Q.loc[solar_panels_condition]

    # Combine the production and emissions data
    solar_panels_data = pd.concat([solar_panels_production, solar_panels_emissions.iloc[[0]]], ignore_index=True)

    # Add the year column
    solar_panels_data['year'] = year

    return solar_panels_data

# Prepare data for all years from 2004 to 2016
all_years_data = {}
for year in range(2004, 2017):
    all_years_data[year] = prepare_data(year)

# Display the data for a specific year for verification
print(all_years_data[2016].head(50))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                         label           0          1            2  \
0   Electricity, Gas and Water  7676.10000  28.147700  8088.370000   
1   Electricity, Gas and Water    -2.61973  -1.801890     4.728270   
2   Electricity, Gas and Water    -2.56368  -1.963460     1.863610   
3   Electricity, Gas and Water    -1.81884  -1.173450     2.038700   
4   Electricity, Gas and Water    -4.12896  -3.024530     3.170070   
5   Electricity, Gas and Water    -2.08869  -1.521290     0.963304   
6   Electricity, Gas and Water   -16.88910 -18.600200     6.703090   
7   Electricity, Gas and Water    -4.08802  -2.983070     2.843810   
8   Electricity, Gas and Water    -1.79493  -1.091510     3.808920   
9   Electricity, Gas and Water    -7.11810  -6.134680    -0.213130   
10  Electricity, Gas and Water    -4.46309  -3.110070     4.513470   
11  Electricity, Gas and Water 

In [ ]:
# Concatenate all yearly data into a single DataFrame
combined_data = pd.concat(all_years_data.values(), ignore_index=True)

# Filter for 'Solar, Tide and Wave Electricity' in the 'label' column
solar_energy_data = combined_data[combined_data['label'].str.contains('Solar, Tide and Wave Electricity', case=False, na=False)]

# Multiply all numeric columns by 1000
solar_energy_data_scaled = solar_energy_data.copy()
numeric_columns = solar_energy_data_scaled.select_dtypes(include=['float64', 'int64']).columns

solar_energy_data_scaled[numeric_columns] = solar_energy_data_scaled[numeric_columns] * 1000

# Filter to keep only columns with non-zero values
non_zero_columns = solar_energy_data_scaled.loc[:, (solar_energy_data_scaled != 0).any(axis=0)]

# Display the filtered data
print(non_zero_columns)

                                 label    0    1    2    3    4    5    6  \
189   Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
379   Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
569   Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
759   Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
949   Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1139  Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1329  Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1519  Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1709  Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1899  Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2089  Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2279  Solar, Tide and Wave Electricity  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

In [ ]:
print(non_zero_columns.sum(axis=0))

label    Solar, Tide and Wave ElectricitySolar, Tide an...
26                                                  1200.0
27                                                  1200.0
28                                                  1200.0
29                                                  1200.0
                               ...                        
4909                                                1200.0
4910                                                1200.0
4911                                                1200.0
4912                                                1200.0
year                                              26130000
Length: 3325, dtype: object
